In [6]:
! pip3 install pandas
! pip3 install evidently
! pip3 install minio
! pip install python-dotenv

In [10]:
from minio import Minio
import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd

from sklearn import datasets

from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab, CatTargetDriftTab

from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection, CatTargetDriftProfileSection 

In [11]:
# Get environment variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')


In [12]:
client = Minio(
    "10.0.105.60:9000",
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    secure=False
)

# Make 'evidently' bucket if not exist.
found = client.bucket_exists("evidently")
if not found:
    client.make_bucket("evidently")
else:
    print("Bucket 'evidently' already exists")

# load CSV Data from MinIO
try:
    client.fget_object('database', 'influxDataLast1Days.csv', './influxDataLast1Days.csv')
    client.fget_object('database', 'influxDataLast30Days.csv', './influxDataLast30Days.csv')
except ResponseError as err:
    print(err)

In [13]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [14]:
# Iris Data
iris_frame = pd.read_csv('influxDataLast30Days.csv')
iris_frame

,name,time,temperature,illuminance,occupancy
0,mqtt_consumer,1646074515218824695,14.43,8874,0
1,mqtt_consumer,1646301576093890487,16.53,19858,1
2,mqtt_consumer,1646301636349450635,16.53,19858,1
3,mqtt_consumer,1646301662030124532,16.53,19858,1
4,mqtt_consumer,1646301673214677252,16.53,19858,1
...,...,...,...,...,...
36637,mqtt_consumer,1647000513110496437,17.68,22155,0
36638,mqtt_consumer,1647000533052303268,17.68,22155,0
36639,mqtt_consumer,1647000552808629609,17.68,22155,0
36640,mqtt_consumer,1647000572679215470,17.68,22155,0


In [15]:
iris_frame['target'] = iris_frame.occupancy

In [16]:
current = int(len(iris_frame.index) * 0.03)
current

1099

In [18]:
#Target and Data Drift Dashboard
iris_data_and_target_drift_dashboard = Dashboard(tabs=[DataDriftTab(verbose_level=0), CatTargetDriftTab(verbose_level=0)])
iris_data_and_target_drift_dashboard.calculate(iris_frame[:current], iris_frame[current:], column_mapping=None)
iris_data_and_target_drift_dashboard.show()

iris_data_and_target_drift_dashboard.save('index.html')
# load HTML file to MinIO
client.fput_object(
    "evidently", "index.html", "./index.html",
)
print(
    "HTML file is successfully uploaded to bucket 'evidently'."
)

HTML file is successfully uploaded to bucket 'evidently'.
